# Extract examples

https://ai.plainenglish.io/harnessing-the-openai-api-with-langchain-and-pydantic-for-structured-data-extraction-30e3e6966699

By leveraging Langchain with Pydantic, developers and businesses alike can enhance their AI-powered applications, making them more robust and efficient. The Pydantic output parser is indeed a powerful tool that unlocks new potentials for structured data extraction from LLMs.


In [1]:
inp = """Large language models (LLMs) have demonstrated remarkable \
generalizability, such as understanding arbitrary entities and relations. \
Instruction tuning has proven effective for distilling LLMs \
into more cost-efficient models such as Alpaca and Vicuna. \
Yet such student models still trail the original LLMs by \
large margins in downstream applications. In this paper, \
we explore targeted distillation with mission-focused instruction \
tuning to train student models that can excel in a broad application \
class such as open information extraction. Using named entity \
recognition (NER) for case study, we show how ChatGPT can be distilled \
into much smaller UniversalNER models for open NER. For evaluation,\
we assemble the largest NER benchmark to date, comprising 43 datasets \
across 9 diverse domains such as biomedicine, programming, social media, \
law, finance. Without using any direct supervision, UniversalNER \
attains remarkable NER accuracy across tens of thousands of entity \
types, outperforming general instruction-tuned models such as Alpaca \
and Vicuna by over 30 absolute F1 points in average. With a tiny \
fraction of parameters, UniversalNER not only acquires ChatGPT's \
capability in recognizing arbitrary entity types, but also \
outperforms its NER accuracy by 7-9 absolute F1 points in average. \
Remarkably, UniversalNER even outperforms by a large margin \
state-of-the-art multi-task instruction-tuned systems such as \
InstructUIE, which uses supervised NER examples. \
We also conduct thorough ablation studies to assess the impact of \
various components in our distillation approach. We will release \
the distillation recipe, data, and UniversalNER models to facilitate \
future research on targeted distillation."""

In [2]:
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

False

In [2]:
import openai
from langchain.chains import create_extraction_chain
from langchain_openai import ChatOpenAI

In [4]:
# Extraction using OpenAI functions
# Schema which will be filled using extracted information

schema = {
    "properties": {
        "research_topic": {"type": "string"},
        "problem_statement": {"type": "string"},
        "experiment_design": {"type": "string"},
        "finding": {"type": "string"},
    },
    "required": ["research_topic", "finding"],
}

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
response = chain.invoke(inp)
print(response)


{'input': "Large language models (LLMs) have demonstrated remarkable generalizability, such as understanding arbitrary entities and relations. Instruction tuning has proven effective for distilling LLMs into more cost-efficient models such as Alpaca and Vicuna. Yet such student models still trail the original LLMs by large margins in downstream applications. In this paper, we explore targeted distillation with mission-focused instruction tuning to train student models that can excel in a broad application class such as open information extraction. Using named entity recognition (NER) for case study, we show how ChatGPT can be distilled into much smaller UniversalNER models for open NER. For evaluation,we assemble the largest NER benchmark to date, comprising 43 datasets across 9 diverse domains such as biomedicine, programming, social media, law, finance. Without using any direct supervision, UniversalNER attains remarkable NER accuracy across tens of thousands of entity types, outperf

## Example 2: Extraction

In [3]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

In [4]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [5]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Person)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatMistralAI.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [7]:
text = "Alan Smith is 6 feet tall and has blond hair."
runnable.invoke({"text": text})

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.8288')